In [1]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.15 µs


In [2]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# パラメータ
n = 4
vm = 0.505
vd = 1.4
ks = 0.5
k1 = 0.5
k2 = 0.6
Km = 0.5
Kd = 0.13
KI = 1
vs_min = 1.6

# vs_max の値とそれに対応する A の計算
vs_max_values = [1.6, 2.25, 3.1, 4.2]
A_values = [(vs_max / vs_min) - 1 for vs_max in vs_max_values]

# vs を時間 t に対する関数として計算する関数
def vs_function(t, A):
    return vs_min * (1 + A * np.sin(t))

# 微分方程式
def model(t, y, A):
    M, FC, FN = y
    vs = vs_function(t, A)  # vs の値を時間 t に応じて計算
    dMdt = vs * (KI**n / (KI**n + FN**n)) - vm * (M / (Km + M))
    dFCdt = ks * M - vd * (FC / (Kd + FC)) - k1 * FC + k2 * FN
    dFNdt = k1 * FC - k2 * FN
    return [dMdt, dFCdt, dFNdt]

# 初期値と時間設定
y0 = [0.5, 0.5, 0.5]
t_span = [0, 2510]
t_eval = np.linspace(t_span[0], t_span[1], 25100)

# 最初の vs_max の値についてのみシミュレーションとプロット
i = 2
A = A_values[i]
sol = solve_ivp(model, t_span, y0, t_eval=t_eval, args=(A,), max_step=0.01)

In [3]:
# CSVファイルにデータを保存するためのDataFrameを作成
vs_values = vs_function(sol.t, A)  # 外力Ap(t)としてvsを計算
data_frame = pd.DataFrame({
    'time': sol.t,
    'M': sol.y[0],
    'FC': sol.y[1],
    'FN': sol.y[2],
    'vs': vs_values  # 外力Ap(t)としての列
})

# CSVファイルにデータを保存
filename = 'LD_cycle1.1.csv'
data_frame.to_csv(filename, index=False)

# CSVファイルを読み込む
data_loaded = pd.read_csv(filename)

# CSVから値を抽出してNumpy配列に格納
X = data_loaded[['M', 'FC', 'vs']].values  

In [4]:
X.shape

(25100, 3)

In [5]:
from hyperopt import hp, tpe, Trials, fmin

In [6]:
# Objective functions accepted by ReservoirPy must respect some conventions:
#  - dataset and config arguments are mandatory, like the empty '*' expression.
#  - all parameters that will be used during the search must be placed after the *.
#  - the function must return a dict with at least a 'loss' key containing the result
# of the loss function. You can add any additional metrics or information with other 
# keys in the dict. See hyperopt documentation for more informations.
def objective(dataset, config, *, iss, N, sr, lr, ridge, seed):
    
    # This step may vary depending on what you put inside 'dataset'
    train_data, validation_data = dataset
    X_train, y_train = train_data
    X_val, y_val = validation_data
    
    # You can access anything you put in the config 
    # file from the 'config' parameter.
    instances = config["instances_per_trial"]
    
    # The seed should be changed across the instances, 
    # to be sure there is no bias in the results 
    # due to initialization.
    variable_seed = seed 
    
    N_value = int(N)  # Nを整数に変換    

    
    losses = []; r2s = [];
    for n in range(instances):
        # Build your model given the input parameters
        reservoir = Reservoir(N_value, 
                              sr=sr, 
                              lr=lr, 
                              input_scaling=iss, 
                              seed=variable_seed)
        
        readout = Ridge(ridge=ridge)

        model = reservoir >> readout


        # Train your model and test your model.
        prediction = model.fit(X_train, y_train) \
                           .run(X_test)
        
        loss = nrmse(y_test, prediction, norm_value=np.ptp(X_train))
        r2 = rsquare(y_test, prediction)
        
        # Change the seed between instances
        variable_seed += 1
        
        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when
    # using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}

In [7]:
hyperopt_config = {
    "exp": f"hyperopt-LD_cycle2.1", # the experimentation name
    "hp_max_evals": 300,             # the number of differents sets of parameters hyperopt has to try
    "hp_method": "tpe",           # the method used by hyperopt to chose those sets (see below)
    "seed": 42,                      # the random state seed, to ensure reproducibility
    "instances_per_trial": 3,        # how many random ESN will be tried with each sets of parameters
    "hp_space": {                    # what are the ranges of parameters explored
        "N": ["quniform", 500, 5000, 1],  # the number of neurons is fixed to 500
        "sr": ["loguniform", 1e-2, 10],   # the spectral radius is log-uniformly distributed between 1e-2 and 10
        "lr": ["loguniform", 1e-3, 1],  # idem with the leaking rate, from 1e-3 to 1
        "iss": ["uniform", 0, 1],           # the input scaling uniformly distributed between 0 and 1
        "ridge": ["loguniform", 1e-9, 1e-2],        # and so is the regularization parameter.
        "seed": ["choice", 5555]          # an other random seed for the ESN initialization
    }
}


import json

# we precautionously save the configuration in a JSON file
# each file will begin with a number corresponding to the current experimentation run number.
with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
    json.dump(hyperopt_config, f)


In [8]:
from reservoirpy.datasets import to_forecasting

train_len = 10000
test_len = 10000

x, y = to_forecasting(X, forecast=1)
X_train, y_train = x[:train_len], y[:train_len]
X_test, y_test = x[train_len:train_len+test_len], y[train_len:train_len+test_len]

dataset = ((X_train, y_train), (X_test, y_test))

In [9]:
from reservoirpy.hyper import research

best = research(objective, dataset, f"{hyperopt_config['exp']}.config.json", ".")

  0%|          | 0/300 [00:00<?, ?trial/s, best loss=?]

  1%|          | 2/300 [05:23<13:32:43, 163.64s/trial, best loss: 0.0002608745841776886]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.24038e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=6.55945e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.33254e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



  1%|▏         | 4/300 [06:53<6:32:10, 79.50s/trial, best loss: 0.00016485203055257474]  

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=8.6476e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=1.10009e-16): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=8.84675e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



...

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=6.41871e-18): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=6.33895e-18): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=6.71536e-18): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



 98%|█████████▊| 294/300 [12:11:04<17:07, 171.28s/trial, best loss: 1.4156957446503661e-08]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=3.21739e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=4.35839e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=3.29557e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



 98%|█████████▊| 295/300 [12:15:18<16:20, 196.08s/trial, best loss: 1.4156957446503661e-08]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.87318e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



In [ ]:
best

({'N': 0,
  'iss': 0.08953586004134119,
  'lr': 0.9765528770413411,
  'ridge': 4.665293050641528e-09,
  'seed': 0,
  'sr': 0.6321052521221111},
 <hyperopt.base.Trials at 0x107c83c40>)

In [ ]:
# `best`タプルの最初の要素には最適化されたハイパーパラメータが直接含まれています
best_params = best[0]

# numpy int64型をPythonのint型に変換するための関数
def convert(o):
    if isinstance(o, np.int64): return int(o)
    raise TypeError

# 最適なハイパーパラメータをJSONファイルに保存
with open(f"{hyperopt_config['exp']}_best_params.json", 'w') as f:
    json.dump(best_params, f, default=convert)